# 🧠 Projeto: Impacto dos Alojamentos Locais no Mercado da Habitação
Este notebook faz parte da Fase 1 do projeto da unidade curricular.

## 📌 Objetivos:
- Integrar dados de diferentes fontes.
- Analisar a relação entre densidade de alojamentos locais, população e rendas
- Implementar técnicas de data cleaning, schema integration e identity resolution

---

## 📦 Importação de Dados

Nesta secção, são importados os diferentes datasets necessários para a análise do impacto do Airbnb em fatores urbanos em Portugal. Cada dataset provém de uma fonte distinta e representa uma perspetiva específica da realidade urbana, como densidade populacional, oferta de alojamentos locais, preços de renda, e dados territoriais.

Os datasets importados são os seguintes:

- **`portugal_listings.csv`**: Contém informações sobre os imóveis disponíveis para aluguer em território nacional, com detalhes como `Price`, `District`, `City` e `Town`.
- **`rendasm2.json`**: Fornece os valores médios das rendas por metro quadrado em diferentes regiões para o ano de **2023**.
- **`densidadePopulacional.json`**: Indica a densidade populacional (Nº habitantes/km²) por concelho para o ano de **2023**.
- **`densidadealojamentosm2.json`**: Apresenta a densidade de alojamentos locais (Nº alojamentos/km²), com granularidade ao nível da freguesia, referente ao ano de **2021**.
- **`Areas_Freg_Conc_Dist_Pais_CAOP2019.xls`**: Ficheiro do CAOP (Carta Administrativa Oficial de Portugal) que define a estrutura administrativa do país, incluindo distritos, concelhos e freguesias. Utilizado como base de referência para unificar códigos geográficos (`DICO`, `geocod`, etc.).

> ⚠️ Cada ficheiro foi lido respeitando a estrutura interna dos formatos `.csv`, `.json` ou `.xls`, e apenas os dados relevantes foram extraídos para posterior limpeza e integração.



In [257]:
import pandas as pd

# Leitura dos datasets
# O dataset listings contém informações sobre os imóveis disponíveis para venda
listings = pd.read_csv(r'dados\portugal_listings.csv')

# O dataset rendas contém informações sobre os preços das rendas por metro quadrado
rendas = pd.read_json(r'dados\rendasm2.json')
rendas = pd.DataFrame(rendas['Dados'][0]['2023'])

# O dataset densidade_pop contém informações sobre a densidade populacional (Nº/km2) por conselho
densidade_pop = pd.read_json(r'dados\densidadePopulacional.json')
densidade_pop = pd.DataFrame(densidade_pop['Dados'][0]['2023'])

# O dataset densidade_aloj contém informações sobre a densidade de alojamentos (Nº/km2) por freguesia
densidade_aloj = pd.read_json(r'dados\densidadealojamentosm2.json')
densidade_aloj = pd.DataFrame(densidade_aloj['Dados'][0]['2021'])

# A tabela municipios contém informações sobre a ordenação territorial portuguesa
concelhos = pd.read_excel(r'dados\Areas_Freg_Conc_Dist_Pais_CAOP2019.xls', sheet_name='Areas_Concelhos_CAOP2019')

Após a importação dos dados, o próximo passo será o data profiling e integration, de modo a perceber as estruturas de dados

---

## 🔍 Data Profiling

Após a importação dos dados, foi realizada uma análise exploratória (data profiling) para compreender a estrutura de cada dataset. Esta análise incluiu:

- Verificação dos tipos de dados;
- Número de valores nulos;
- Percentagem de nulos;
- Cardinalidade (valores únicos) por coluna.

A função usada para este processo foi:


In [258]:
def DataProfiling(dataframe: pd.DataFrame):
    # Data profiling completo para todas as colunas
    data_profiling = pd.DataFrame({
        "Coluna": dataframe.columns,
        "Tipo de Dado": dataframe.dtypes.values,
        "Valores Não Nulos": dataframe.notnull().sum().values,
        "Valores Nulos": dataframe.isnull().sum().values,
        "% Nulos": (dataframe.isnull().mean() * 100).round(2).values,
        "Valores Únicos": dataframe.nunique().values
    })
    display(data_profiling)  

### Areas_Freg_Conc_Dist_Pais_CAOP2019.xls



In [259]:
DataProfiling(concelhos)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,DICO,int64,308,0,0.0,308
1,NUTSI_DSG,object,308,0,0.0,3
2,NUTSI_COD,int64,308,0,0.0,3
3,NUTSII_DSG,object,308,0,0.0,7
4,NUTSII_COD,int64,308,0,0.0,7
5,NUTSIII_DSG,object,308,0,0.0,25
6,NUTSIII_COD,object,308,0,0.0,25
7,DISTRITO_ILHA_DSG,object,308,0,0.0,29
8,CONCELHO_DSG,object,308,0,0.0,307
9,AREA_2019_ha,float64,308,0,0.0,308


⚙️ **Seleção de Colunas Relevantes — `Areas_Concelhos_CAOP2019`**

Contém 14 colunas, das quais apenas 3 foram consideradas relevantes:

- `DICO`: Identificador único do concelho (**manter**).
- `CONCELHO_DSG`: Nome oficial do concelho (**manter**).
- `NUTSIII_DSG`: Região NUTS III (**manter**).

As colunas a eliminar incluem:
- `NUTSI_DSG`, `NUTSI_COD`, `NUTSII_DSG`, `NUTSII_COD`, `NUTSIII_COD`: não necessárias à análise.
- `DISTRITO_ILHA_DSG`: nível distrital, não será utilizado.
- `AREA_2019_ha`, `AREA_2019_km2`, `PERIM_km`, `ALTITUDE_MAX_m`, `ALTITUDE_MIN_m`: não relevantes para o estudo.



### densidadealojamentosm2.json



In [260]:
DataProfiling(densidade_aloj)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,3439,0,0.0,3439
1,geodsg,object,3439,0,0.0,3075
2,ind_string,object,3439,0,0.0,1656
3,valor,object,3439,0,0.0,1656


⚙️ **Seleção de Colunas Relevantes — `densidadealojamentosm2`**

Contém 4 colunas, sendo úteis:

- `geocod`: Identificador único da freguesia (**manter**).
- `geodsg`: Nome oficial da freguesia (**manter**).
- `valor`: Valor do indicador (**manter**, a converter para `float`).

A coluna `ind_string` é redundante e será **eliminada**.

### DensidadePopulacional.json



In [261]:
DataProfiling(densidade_pop)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,347,0,0.0,347
1,geodsg,object,347,0,0.0,340
2,ind_string,object,347,0,0.0,316
3,valor,object,347,0,0.0,316


⚙️ **Seleção de Colunas Relevantes — `DensidadePopulacional`**

Com estrutura semelhante ao anterior. Colunas relevantes:

- `geocod`: Identificador do concelho (**manter**).
- `geodsg`: Nome oficial do concelho (**manter**).
- `valor`: Valor do indicador (**manter**, a converter para `float`).

`ind_string` será **eliminado**.




### portugal_listings.csv

In [262]:
DataProfiling(listings)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,Price,float64,135236,300,0.22,4754
1,District,object,135536,0,0.00,27
2,City,object,135536,0,0.00,275
3,Town,object,135534,2,0.00,2263
4,Unnamed: 4,float64,0,135536,100.00,0
5,Unnamed: 5,float64,0,135536,100.00,0
6,Unnamed: 6,float64,0,135536,100.00,0
7,Unnamed: 7,float64,0,135536,100.00,0
8,Unnamed: 8,float64,0,135536,100.00,0
9,Unnamed: 9,float64,0,135536,100.00,0


⚙️ **Seleção de Colunas Relevantes — `portugal_listings`**

Contém 14 colunas. Apenas 3 foram consideradas essenciais:

- `Price`: Preço do imóvel (**manter**).
- `District`: Nome do distrito (**manter**).
- `City`: Nome da cidade (**manter**).

As colunas `Town` e `Unnamed: 4` a `Unnamed: 13` serão **eliminadas** por irrelevância e redundância, respetivamente.



### rendasm2.json


In [263]:
DataProfiling(rendas)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,638,0,0.00,638
1,geodsg,object,638,0,0.00,616
2,ind_string,object,638,0,0.00,343
3,valor,object,431,207,32.45,341
4,sinal_conv,object,207,431,67.55,2
5,sinal_conv_desc,object,207,431,67.55,2


⚙️ **Seleção de Colunas Relevantes — `rendasm2`**

Contém 6 colunas. As seguintes foram mantidas:

- `geocod`: Identificador do concelho (**manter**).
- `geodsg`: Nome do concelho (**manter**).
- `valor`: Valor médio da renda (**manter**, a converter para `float`).

As colunas `ind_string`, `sinal_conv`, `sinal_conv_desc` foram consideradas auxiliares e serão **eliminadas**.



### 📊 Conclusões do Data Profiling

Após a leitura e análise dos datasets, foram identificadas as seguintes características e decisões de limpeza e preparação:


#### 📌 Colunas-Chave Identificadas

| Dataset                     | Chave Primária                              | Chave(s) Secundária(s)         |
|----------------------------|---------------------------------------------|--------------------------------|
| `Areas_Concelhos_CAOP2019` | `DICO` → renomeado para `MunicipalityCode` | `CONCELHO_DSG`, `NUTSIII_DSG` |
| `densidadePopulacional`    | `geocod` (últimos 4 dígitos) → `MunicipalityCode` | `geodsg`             |
| `densidadealojamentosm2`   | `geocod` (primeiros 4 dígitos) → `MunicipalityCode` | `geodsg`            |
| `rendasm2`                 | `geocod` (últimos 4 dígitos) → `MunicipalityCode` | `geodsg`             |
| `portugal_listings`        | `City`                                      | `District`, `Price`             |

⚠️ Nota: `geocod` foi mapeado para `MunicipalityCode` com `str[:4]` ou `str[-4:]` conforme o dataset, garantindo uniformidade nas junções.


#### 🔍 Conversões e Limpezas Realizadas

- As colunas `valor` foram convertidas para `float` com `pd.to_numeric(errors='coerce')`.
- Os registos com valores **nulos em `Price`** (no dataset `listings`) foram **removidos**.
- Os valores nulos nas colunas `valor` dos datasets **`rendas`** foram **substituídos pela média do distrito**.
- Todas as colunas geográficas foram **normalizadas**:
  - Remoção de acentos;
  - Conversão para minúsculas;
  - Remoção de espaços em branco;
  - Criação de colunas auxiliares como `*_norm`.


#### 🧱 Diferença de Granularidade

- Os datasets `densidadealojamentosm2` e `rendasm2` estão ao nível da **freguesia**.
- Os restantes datasets estão ao nível do **concelho** (`MunicipalityCode`).

🔄 **Solução**:
- Realizar um **agrupamento por concelho** (usando os primeiros ou últimos 4 dígitos de `geocod`).
- As **médias por concelho** serão calculadas para:
  - Densidade de alojamentos locais;
  - Preço das rendas por m².
- Isso garante coerência na granularidade entre os diferentes conjuntos de dados para posterior integração e análise.


---

## 🧽 Data Cleaning & Preparation

Após a fase de **Data Profiling**, foi possível identificar várias inconsistências nos datasets utilizados neste projeto. Esta fase de **Data Cleaning & Integration** tem como objetivo preparar os dados para a integração e análise final, assegurando que todas as fontes de dados partilham uma estrutura coerente e limpa.


### 🔍 Problemas Identificados

- Colunas com valores numéricos armazenados como `string`;
- Utilização inconsistente de separadores decimais (`"."` e `","`);
- Existência de **valores nulos** em campos críticos como `Price` e `valor`;
- Presença de acentos, letras maiúsculas/minúsculas e espaços inconsistentes em nomes geográficos (`District`, `Town`, `City`, `geodsg`, etc.);
- Diferença de granularidade entre datasets (ex: freguesia vs concelho).



### 🎯 Objetivos da Limpeza e Preparação

1. **Conversão de colunas numéricas** para o tipo `float`, garantindo a coerência dos dados;
2. **Tratamento de valores nulos**, através de:
   - Remoção em colunas como `Price`;
   - Substituição pela **média distrital** no dataset com `valor`, `rendasm2`;
3. **Normalização dos nomes de localidades** para garantir a consistência entre datasets:
   - Remoção de acentos;
   - Conversão para minúsculas;
   - Eliminação de espaços em branco;
4. **Criação de colunas normalizadas (`*_norm`)**, que servirão como **chaves secundárias** durante a integração dos dados;
5. **Harmonização das chaves primárias**, como `MunicipalityCode`, obtidas a partir do `geocod` (início ou fim da string, conforme o dataset);
6. **Preparação de DataFrames tratados**, com apenas as colunas essenciais para futura análise.


Com estas operações de limpeza, os datasets ficam prontos para a próxima etapa do projeto: **Schema Integration e Identity Resolution**.


In [264]:
!pip install unidecode


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Função para normalizar os nomes:

In [265]:
import unidecode

# Normalizar nomes
def normalizar(texto):
    return unidecode.unidecode(str(texto).strip().lower())

#### Areas_Freg_Conc_Dist_Pais_CAOP2019

Este ficheiro do CAOP fornece a divisão administrativa oficial de Portugal, com hierarquias de freguesia, concelho e distrito. A folha Areas_Concelhos_CAOP2019 foi selecionada como base de referência para garantir consistência geográfica durante a integração dos dados.
- Inclui diversas colunas administrativas, nem todas relevantes para a análise;
- Os nomes dos concelhos e regiões precisam de ser normalizados;
- É necessário renomear colunas para maior clareza e coerência com os restantes datasets.

✅ Objetivos da Limpeza:
- Selecionar apenas colunas relevantes: `DICO`, `CONCELHO_DSG`, `NUTSIII_DSG`;
- Renomear colunas para `MunicipalityCode`, `Municipality`, `Region`;
- Normalizar nomes (`Municipality`, `Region`);
- Exportar dataset tratado como referência para junções (concelhosTratado.csv).

 1. Selecionar Colunas Relevantes

In [266]:
dfConcelhos = pd.DataFrame(concelhos)

colunasParaManter = ['DICO', 'CONCELHO_DSG', 'NUTSIII_DSG']
dfConcelhos = dfConcelhos[colunasParaManter]

2. Renomear Colunas com Nomes Descritivos

In [267]:
dfConcelhos = dfConcelhos.rename(columns={
    'DICO': 'MunicipalityCode',
    'CONCELHO_DSG': 'Municipality',
    'NUTSIII_DSG': 'Region'
})

 3. Normalizar Colunas de Texto

In [268]:
dfConcelhos['MunicipalityCode'] = dfConcelhos['MunicipalityCode'].astype(str).str.zfill(4)
dfConcelhos['Municipality_norm'] = dfConcelhos['Municipality'].apply(normalizar)
dfConcelhos['Region_norm'] = dfConcelhos['Region'].apply(normalizar)

> As colunas `Municipality_norm` e `Region_norm` serão usadas como auxilary keys nos joins entre datasets.

4. Exportar Tabela Tratada

In [269]:
dfConcelhosTratado = dfConcelhos[[
    'MunicipalityCode', 'Municipality', 'Municipality_norm',
    'Region', 'Region_norm'
]].drop_duplicates().copy()

dfConcelhosTratado.to_csv(r'dados\dados tradados/concelhosTratado.csv', index=False)

> A tabela `concelhosTratado.csv` será usada como base de referência para integrar os dados das rendas, densidade populacional, alojamentos e listings.

#### densidadealojamentosm2

Este dataset indica a densidade de alojamentos locais por km² ao nível da freguesia. Por estar numa granularidade inferior, é necessário ajustá-lo para concelho, a fim de permitir comparações consistentes com os restantes dados.
- Os dados estão organizados por geocod, que representa freguesias;
- Existem valores nulos em algumas entradas da densidade;
- Os nomes (`geodsg`) requerem normalização para integração.

✅ Objetivos da Limpeza:
- Extrair o código do concelho (`MunicipalityCode`) a partir do `geocod`;
- Normalizar os nomes das freguesias (`geodsg`);
- Substituir valores nulos pela média do concelho;
- Agregar a densidade média por concelho;
- Adicionar colunas normalizadas para integração.

1. Carregar Dados e Extrair Código do Concelho

In [270]:
dfAloj = pd.DataFrame(densidade_aloj)

# 🧾 Extrair código do concelho (4 primeiros dígitos do geocod)
dfAloj['MunicipalityCode'] = dfAloj['geocod'].str[:4]

2. Juntar com Concelhos (para nomes normalizados)

In [271]:
# 🔁 Juntar com concelhos para obter nomes e norm
dfAlojMerged = dfAloj.merge(
    dfConcelhos[['MunicipalityCode', 'Municipality', 'Municipality_norm']],
    on='MunicipalityCode', how='left'
)

3. Filtrar Matches Válidos

In [272]:
# Filtrar apenas matches válidos
alojamentosMatch = dfAlojMerged[dfAlojMerged['Municipality'].notna()].copy()

 4. Converter Densidade para float e Substituir Nulos

In [273]:
# Converter densidade para float
alojamentosMatch['Density'] = pd.to_numeric(alojamentosMatch['valor'], errors='coerce')

# Substituir valores nulos pela média do concelho
alojamentosMatch['Density'] = alojamentosMatch.groupby('MunicipalityCode')['Density'].transform(
    lambda x: x.fillna(x.mean())
)

5. Agregar Freguesias por Concelho (tratar da granularidade)

In [274]:
densidadeAlojMedia = (
    alojamentosMatch
    .groupby(['MunicipalityCode', 'Municipality', 'Municipality_norm'])
    .agg({'Density': 'mean'})
    .reset_index()
    .rename(columns={'Density': 'DensityAlojMean'})
)

6. Exportar Dataset Tradato

In [275]:
densidadeAlojMedia.to_csv(r'dados\dados tradados/densidadeAlojTratado.csv', index=False)

> A tabela final `densidadeAlojTratado.csv` contém a média da densidade de alojamentos por concelho, pronta para integração com os restantes datasets.

#### DensidadePopulacional

Este dataset apresenta a densidade populacional (número de habitantes por km²) por concelho, com dados já estruturados ao nível municipal, o que facilita a sua integração com os restantes datasets.
- Já inclui o código do concelho (`geocod`);

- Os nomes das regiões (`geodsg`) necessitam de normalização;

- A coluna valor deve ser convertida para numérica.

✅ Objetivos da Limpeza:

- Extrair `MunicipalityCode` a partir do `geocod`;

- Normalizar o nome da unidade geográfica (`geodsg`);

- Converter a densidade (`valor`) para float;

- Gerar colunas auxiliares (`Municipality`, `Municipality_norm`);

- Preparar o dataset final sem necessidade de agregação.

1. Extrair código do concelho (últimos 4 dígitos do geocod)

In [276]:
dfPop = pd.DataFrame(densidade_pop)

# 🧾 Extrair código do concelho (últimos 4 dígitos do geocod)
dfPop['MunicipalityCode'] = dfPop['geocod'].str[-4:]

2. Normalizar nome da unidade geográfica (`geodsg` → `Municipality_norm`)

In [277]:
dfPop['Municipality_norm'] = dfPop['geodsg'].apply(normalizar)
dfPop.rename(columns={'geodsg': 'Municipality'}, inplace=True)

3. Converter `valor` para float → coluna `Density`

In [278]:
dfPop['Density'] = pd.to_numeric(dfPop['valor'], errors='coerce')

4. Exportar CSV final tratado

In [279]:
dfPopTratado = dfPop[['geocod', 'Municipality', 'Municipality_norm', 'Density']]

# 💾 Exportar CSV final
dfPopTratado.to_csv(r'dados\dados tradados/densidadePopTratado.csv', index=False)


#### portugal_listings

Este dataset contém informações sobre os imóveis anunciados para aluguer em território nacional. Apesar de ser o dataset mais importante para o estudo do impacto do alojamento local, apresenta estrutura inconsistente com os restantes datasets, nomeadamente:
- Falta de código administrativo (geocod);
- Nomes de localidades em diferentes formatos (`District`, `City`, `Town`);
- Registos com preços inválidos (nulos ou 0).

✅ Objetivos da Limpeza:
- Remover colunas irrelevantes (`Unnamed`, `Town`);
- Converter Price para float, tratando entradas inválidas;
- Eliminar registos com preços ausentes ou inferiores a 1€;
- Normalizar os nomes das localidades (`District` e `City`);
- Substituir `City` por `Municipality`, para manter coerência com os restantes datasets;
- Gerar colunas auxiliares `*_norm` para usar como chaves durante o blocking e resolução de identidade.
> 🧠 Nesta fase, ainda não é possível adicionar `MunicipalityCode` ou `geocod`, visto que o dataset listings carece desta informação. A resolução de identidade com `dfConcelhosTratado` será feita na fase seguinte, através de blocking e similaridade textual.

1. Remover colunas irrelevantes

In [280]:
dfListings = pd.DataFrame(listings)

# 🧹 Remover colunas 'Unnamed' e 'Town' (não será usada)
dfListings = dfListings.drop(columns=[col for col in dfListings.columns if "Unnamed" in col])
dfListings = dfListings.drop(columns=['Town'], errors='ignore')

2. Converter `Price` para float e remover linhas com `Price` nulo ou menor que 0

In [281]:
# Converter 'Price' para float
dfListings['Price'] = pd.to_numeric(dfListings['Price'], errors='coerce')

# Filtrar linhas com preço válido (> 0)
dfListings = dfListings[dfListings['Price'].notna() & (dfListings['Price'] > 0)].copy()


3. Normalizar as colunas de texto e rename à coluna `City` para `Municipality`

In [282]:

dfListings['District_norm'] = dfListings['District'].apply(normalizar)
dfListings['Municipality'] = dfListings['City']  # Substitui City por Municipality
dfListings['Municipality_norm'] = dfListings['Municipality'].apply(normalizar)
dfListings = dfListings.drop(columns=['City'])

4. Exportar dados para tabela tradata

In [283]:
# 💾 Exportar CSV final tratado
dfListings.to_csv(r'dados\dados tradados/listingsTratado.csv', index=False)

#### rendasm2

Este dataset disponibiliza os valores médios das rendas por m<sup>2</sup> em Portugal, com granularidade ao nível da freguesia. É uma fonte crucial para entender os padrões de custo de habitação e a sua variação regional. No entanto, apresenta algumas limitações estruturais:
- Está ao nível da freguesia, sendo necessário agregá-lo ao nível do concelho;
- Algumas entradas têm valores nulos na coluna valor;
- Os nomes das regiões (coluna `geodsg`) podem apresentar variações e inconsistências linguísticas.

✅ Objetivos da Limpeza:
- Converter a coluna valor para o tipo float;
- Extrair o código do concelho (`MunicipalityCode`) a partir do geocod;
- Normalizar os nomes geográficos (`geodsg`);
- Substituir valores nulos pela média do concelho;
- Agregar os valores médios de renda ao nível municipal;
- Incluir colunas `Municipality` e `Municipality_norm` para integração.

1. Converter `valor` para float

In [284]:
dfRendas = pd.DataFrame(rendas)

# Converter 'valor' para float
dfRendas['RentValue'] = pd.to_numeric(dfRendas['valor'], errors='coerce')

2. Extrair código do concelho e normalizar nome

In [285]:

dfRendas['MunicipalityCode'] = dfRendas['geocod'].str[-4:]

# Normalizar nome do município
dfRendas['MunicipalityNorm'] = dfRendas['geodsg'].apply(normalizar)

3. Juntar com concelhos para validar correspondências

In [286]:
# Juntar com concelhos apenas para validar matches
rendasMerged = dfRendas.merge(
    dfConcelhos[['MunicipalityCode', 'Municipality', 'Municipality_norm']],
    on='MunicipalityCode', how='left'
)

4. Filtrar apenas correspondências válidas

In [287]:
# filtrar apenas matches válidos
rendasMatch = rendasMerged[rendasMerged['Municipality'].notna()].copy()

5. Substituir valores nulos pela média do concelho

In [288]:
# Substituir valores nulos pela média por concelho
rendasMatch['RentValue'] = rendasMatch.groupby('MunicipalityCode')['RentValue'].transform(
    lambda x: x.fillna(x.mean())
)

6. Agregar valor médio por concelho

In [289]:
# Agregar média por concelho
rendasPorConcelho = (
    rendasMatch
    .groupby('MunicipalityCode')
    .agg({'RentValue': 'mean'})
    .reset_index()
    .rename(columns={'RentValue': 'RentValueMean'})
)

7. Adicionar nomes normalizados ao resultado final

In [290]:
# Adicionar Municipality e MunicipalityNorm
rendasPorConcelho = rendasPorConcelho.merge(
    dfConcelhos[['MunicipalityCode', 'Municipality', 'Municipality_norm']],
    on='MunicipalityCode', how='left'
)

8. Exportar CSV final tratado

In [291]:
dfRendas = pd.DataFrame(rendasPorConcelho)

# Exportar CSV final
dfRendas.to_csv(r'dados\dados tradados/rendasTratado.csv', index=False)

---

## 🔗 Plano de Integração de Esquemas

Após o processo de limpeza e normalização dos dados, foi definido o seguinte plano para integrar os diversos datasets, tendo em conta as diferenças de granularidade e estrutura dos mesmos.

### 🧩 Objetivo da Integração

Integrar os datasets a um **nível municipal (MunicipalityCode)** para permitir a análise cruzada entre:
- Preços das rendas por m² (`rendasm2`)
- Densidade populacional (`densidadePopulacional`)
- Densidade de alojamentos locais (`densidadeAlojamentosm2`)
- Preços dos imóveis (`portugal_listings`)
- Localização administrativa (`Areas_Concelhos_CAOP2019`)


### 🗂️ Granularidade dos Dados

| Dataset                | Granularidade Original | Estratégia de Uniformização       |
|------------------------|------------------------|-----------------------------------|
| `densidadeAlojamentos` | Freguesia              | Agrupar por concelho com a média  |
| `rendasm2`             | Freguesia              | Agrupar por concelho com a média  |
| `densidadePopulacional`| Concelho               | Já está no nível correto          |
| `portugal_listings`    | Freguesia              | Usar a coluna concelho            |
| `Areas_Concelhos_CAOP2019` | Concelho           | Base de referência administrativa |


### 🧱 Chaves

- A **chave de integração principal** será `MunicipalityCode`, extraída de `geocod`, `DICO`, ou derivada de `City`.
- Para garantir consistência, foi criada a coluna `Municipality_norm` (nomes normalizados de concelhos).
- Os nomes de cidades (`City`) serão normalizados e mapeados para os concelhos correspondentes manualmente ou via regra de decisão (ex: correspondência direta com `Municipality_norm`).


### 📋 Estratégia de Integração

1. **Densidade de Alojamentos**:  
   - Agregar por `MunicipalityCode` (média dos valores da freguesia).
   - Gerar dataset final com: `MunicipalityCode`, `DensityAlojMean` e `Municipality_norm`.

2. **Densidade Populacional**:  
   - Já vem por `MunicipalityCode`, apenas normalizar e confirmar correspondência.

3. **Rendas por m²**:  
   - Agregar por `MunicipalityCode` (média dos valores da freguesia).
   - Gerar dataset com: `MunicipalityCode`, `RentValueMean` e `Municipality_norm`.

4. **Listings**:  
   - Associar `Municipality_norm` a `Municipality_norm` da tabela `concelhosTratado.csv` e obter `MunicipalityCode`.
   - Calcular estatísticas agregadas (média de preço, contagem) por concelho.


### 🧠 Considerações Finais

- O dataset `concelhosTratado.csv` será a **tabela base** para todas as junções.
- As colunas `MunicipalityCode` e `Municipality_norm` serão usadas como **blocking keys**.
- Pode ser necessário fazer **resolução de identidade** entre nomes de cidades (`City`) e concelhos (`Municipality`) para garantir boas ligações entre listings e os restantes datasets.

---

## 🧱 Blocking Strategy & Similarity Metrics

Durante a resolução de identidade e planeamento da integração, é essencial preparar mecanismos que reduzam o custo computacional das comparações entre registos (blocking) e definam critérios objetivos para medir semelhança (similarity metrics).

### 📦 Blocking Strategy

**Objetivo:** Reduzir o número de comparações entre entidades de diferentes datasets, criando "blocos" onde a comparação faz sentido (ex: localidades com o mesmo prefixo ou região).

#### Estratégia adotada:
- Criar blocos com base no prefixo do nome normalizado (`Municipality_norm`).
- Apenas comparamos nomes dentro do mesmo bloco.

### 📏 Similarity Metrics

Após o processo de blocking, é necessário aplicar métricas de similaridade para avaliar o quão próximos são dois nomes normalizados (`Municipality_norm`) de diferentes datasets. Para isso, foram utilizadas três métricas distintas da biblioteca `RapidFuzz`, que abordam a comparação textual sob diferentes perspetivas.

#### ✅ Métricas Utilizadas

| Métrica                   | Descrição                                                                 | Vantagens                                                                 |
|---------------------------|---------------------------------------------------------------------------|---------------------------------------------------------------------------|
| `token_sort_ratio`        | Ordena alfabeticamente as palavras antes da comparação                   | Lida bem com nomes invertidos, como `"porto vila"` vs `"vila porto"`      |
| `ratio` (Levenshtein)     | Mede a distância de edição entre duas strings                            | Capta pequenas variações e erros de digitação                             |
| `token_set_ratio`         | Compara os conjuntos únicos de palavras, ignorando repetições e ordem    | Útil quando um nome é subconjunto do outro: `"santa maria da feira"` vs `"feira"` |

Estas métricas são complementares e foram escolhidas por permitirem capturar:
- Similaridade estrutural (`token_sort_ratio`);
- Variações simples de escrita (`ratio`);
- Inclusões e subconjuntos de palavras (`token_set_ratio`).

> 📌 A utilização combinada destas métricas aumenta a robustez da correspondência entre nomes, facilitando a resolução de identidade.

In [292]:
!pip install rapidfuzz


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 1. ✏️ Comparação 1: `listings` vs `rendas`

1.1 Carregar os datasets tratados

In [293]:
import pandas as pd
from rapidfuzz import fuzz

dfListings = pd.read_csv('dados/dados tradados/listingsTratado.csv')
dfRendas = pd.read_csv('dados/dados tradados/rendasTratado.csv')

> 🧾 Objetivo: Importar os dois datasets que serão comparados — os listings e os valores de renda (rendas). Estes já foram limpos e normalizados anteriormente, incluindo a criação da coluna `Municipality_norm`.

1.2 Criar chave de blocking

In [294]:
dfListings['block_key'] = dfListings['Municipality_norm'].str[:4]
dfRendas['block_key'] = dfRendas['Municipality_norm'].str[:4]

> 🔍 Objetivo: Reduzir o número de comparações utilizando uma chave de blocking.
>- A chave `block_key` é formada pelos primeiros 4 caracteres da string normalizada (`Municipality_norm`).
>- Assim, apenas localidades com prefixos semelhantes serão comparadas, o que melhora a performance e reduz falsos positivos.

1.3 Gerar pares candidatos com base no bloco

In [295]:
candidatos = pd.merge(
    dfListings[['Municipality', 'Municipality_norm', 'block_key']],
    dfRendas[['Municipality', 'Municipality_norm', 'block_key']],
    on='block_key',
    suffixes=('_listings', '_rendas')
)

> 🔗 Objetivo: Criar todas as combinações possíveis dentro de cada bloco.
>- Une as duas tabelas pelos block_key.
>- O resultado é um conjunto de pares candidatos a representar a mesma entidade (concelho), mesmo que com variações no nome.

1.4 Calcular as 3 medidas de similaridade

In [296]:
candidatos['similaridade_token_sort'] = candidatos.apply(
    lambda row: fuzz.token_sort_ratio(row['Municipality_norm_listings'], row['Municipality_norm_rendas']) / 100,
    axis=1
)

candidatos['similaridade_levenshtein'] = candidatos.apply(
    lambda row: fuzz.ratio(row['Municipality_norm_listings'], row['Municipality_norm_rendas']) / 100,
    axis=1
)

candidatos['similaridade_token_set'] = candidatos.apply(
    lambda row: fuzz.token_set_ratio(row['Municipality_norm_listings'], row['Municipality_norm_rendas']) / 100,
    axis=1
)

> 📐 Objetivo: Avaliar o grau de semelhança entre os nomes dos concelhos nos dois datasets usando 3 métricas:
> -    `token_sort_ratio`: ordena as palavras antes de comparar, útil quando a ordem das palavras varia.
> -    `ratio (Levenshtein)`: mede a distância de edição entre as strings.
> -    `token_set_ratio`: ignora palavras repetidas e compara subconjuntos únicos, ideal quando há muitos termos comuns.
> Valores de `0` (menos similar) a `1` (mais similar).

1.5 Visualizar os pares com maior similaridade

In [297]:
top_pares = candidatos.sort_values(by='similaridade_token_sort', ascending=False)

display(top_pares[[
    'Municipality_listings', 'Municipality_rendas',
    'similaridade_token_sort',
    'similaridade_levenshtein',
    'similaridade_token_set'
]])

,Municipality_listings,Municipality_rendas,similaridade_token_sort,similaridade_levenshtein,similaridade_token_set
471810,Valpaços,VALPAÇOS,1.000000,1.000000,1.000000
0,Valpaços,VALPAÇOS,1.000000,1.000000,1.000000
471790,Moita,MOITA,1.000000,1.000000,1.000000
471789,Barreiro,BARREIRO,1.000000,1.000000,1.000000
471786,Montijo,MONTIJO,1.000000,1.000000,1.000000
...,...,...,...,...,...
82394,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424
88687,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424
95242,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424
75912,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424


> Objetivo: Ordenar os pares por similaridade mais alta (com base em `token_sort_ratio`) e apresentar os resultados mais promissores.
> - Permite analisar os pares com maior probabilidade de corresponderem à mesma entidade — essencial para fazer identity resolution entre `Municipality` do listings e `Municipality` dos valores de renda.

#### 2. ✏️ Comparação 2: `listings` vs `densidadePop`

2.1 Carregar ps datasets tratados

In [298]:
dfListings = pd.read_csv('dados/dados tradados/listingsTratado.csv')
dfPop = pd.read_csv('dados/dados tradados/densidadePopTratado.csv')


> 🧾 **Objetivo:** Importar os datasets tratados para análise — neste caso, os listings e a densidade populacional (`densidadePop`).  
> Ambos foram previamente limpos e normalizados, incluindo a coluna `Municipality_norm`.


2.2 Criar chave de blocking

In [299]:
dfListings['block_key'] = dfListings['Municipality_norm'].str[:4]
dfPop['block_key'] = dfPop['Municipality_norm'].str[:4]

> 🔍 **Objetivo:** Reduzir o número de comparações com base em blocos.  
> - A chave `block_key` é extraída dos primeiros 4 caracteres da `Municipality_norm`.  
> - Isso permite comparar apenas localidades com nomes semelhantes, reduzindo falsos positivos.

2.3 Gerar pares candidatos com base no bloco

In [300]:
candidatosPop = pd.merge(
    dfListings[['Municipality', 'Municipality_norm', 'block_key']],
    dfPop[['Municipality', 'Municipality_norm', 'block_key']],
    on='block_key',
    suffixes=('_listings', '_pop')
)

> 🔗 **Objetivo:** Criar pares candidatos dentro de cada bloco.  
> - Junta ambos os datasets através do `block_key`.  
> - Resulta em pares com potencial para corresponder à mesma entidade.

2.4 Calcular as 3 medidas de similaridade

In [301]:
candidatosPop['similaridade_token_sort'] = candidatosPop.apply(
    lambda row: fuzz.token_sort_ratio(row['Municipality_norm_listings'], row['Municipality_norm_pop']) / 100,
    axis=1
)

candidatosPop['similaridade_levenshtein'] = candidatosPop.apply(
    lambda row: fuzz.ratio(row['Municipality_norm_listings'], row['Municipality_norm_pop']) / 100,
    axis=1
)

candidatosPop['similaridade_token_set'] = candidatosPop.apply(
    lambda row: fuzz.token_set_ratio(row['Municipality_norm_listings'], row['Municipality_norm_pop']) / 100,
    axis=1
)

> 📐 **Objetivo:** Avaliar semelhança textual entre os nomes das localidades.  
> - `token_sort_ratio`: útil se a ordem das palavras varia.  
> - `ratio`: mede a distância de edição.  
> - `token_set_ratio`: ideal quando há muitos termos comuns entre os nomes.

2.5 Visualizar os pares com maior similaridade

In [302]:
top_pares_pop = candidatosPop.sort_values(by='similaridade_token_sort', ascending=False)

display(top_pares_pop[[
    'Municipality_listings', 'Municipality_pop',
    'similaridade_token_sort',
    'similaridade_levenshtein',
    'similaridade_token_set'
]])

,Municipality_listings,Municipality_pop,similaridade_token_sort,similaridade_levenshtein,similaridade_token_set
494913,Valpaços,Valpaços,1.000000,1.000000,1.000000
0,Valpaços,Valpaços,1.000000,1.000000,1.000000
494894,Montalegre,Montalegre,1.000000,1.000000,1.000000
494893,Moita,Moita,1.000000,1.000000,1.000000
494892,Barreiro,Barreiro,1.000000,1.000000,1.000000
...,...,...,...,...,...
161261,Santo Tirso,Santa Cruz da Graciosa,0.242424,0.424242,0.242424
161077,Santo Tirso,Santa Cruz da Graciosa,0.242424,0.424242,0.242424
134729,Santo Tirso,Santa Cruz da Graciosa,0.242424,0.424242,0.242424
360334,Santo Tirso,Santa Cruz da Graciosa,0.242424,0.424242,0.242424


> 📊 **Objetivo:** Mostrar os pares com maior probabilidade de corresponderem à mesma entidade (`Municipality`).  
> Essencial para ligar listings à densidade populacional ao nível do concelho.

#### 3. ✏️ Comparação 2: `listings` vs `densidadeAloj`

3.1 Carregar os datasets tratados

In [303]:
dfListings = pd.read_csv('dados/dados tradados/listingsTratado.csv')
dfAloj = pd.read_csv('dados/dados tradados/densidadeAlojTratado.csv')

> 🧾 **Objetivo:** Importar os datasets normalizados — listings e densidade de alojamentos locais.

3.2 Criar chave de blocking

In [304]:
dfListings['block_key'] = dfListings['Municipality_norm'].str[:4]
dfAloj['block_key'] = dfAloj['Municipality_norm'].str[:4]

> 🔍 **Objetivo:** Aplicar blocking para limitar as comparações a pares com prefixo semelhante.  
> Reduz o custo computacional e melhora a precisão da comparação.

3.3 Gerar pares candidatos com base no bloco

In [305]:
candidatosAloj = pd.merge(
    dfListings[['Municipality', 'Municipality_norm', 'block_key']],
    dfAloj[['Municipality', 'Municipality_norm', 'block_key']],
    on='block_key',
    suffixes=('_listings', '_aloj')
)

> 🔗 **Objetivo:** Obter pares candidatos entre listings e densidade de alojamento.  
> Os pares são agrupados com base na chave de blocking.

3.4 Calcular as 3 medidas de similaridade

In [306]:
candidatosAloj['similaridade_token_sort'] = candidatosAloj.apply(
    lambda row: fuzz.token_sort_ratio(row['Municipality_norm_listings'], row['Municipality_norm_aloj']) / 100,
    axis=1
)

candidatosAloj['similaridade_levenshtein'] = candidatosAloj.apply(
    lambda row: fuzz.ratio(row['Municipality_norm_listings'], row['Municipality_norm_aloj']) / 100,
    axis=1
)

candidatosAloj['similaridade_token_set'] = candidatosAloj.apply(
    lambda row: fuzz.token_set_ratio(row['Municipality_norm_listings'], row['Municipality_norm_aloj']) / 100,
    axis=1
)

> 📐 **Objetivo:** Calcular a similaridade entre nomes para facilitar a correspondência automática.  
> As três métricas permitem complementar a análise em diferentes cenários de variação textual.

3.5 Visualizar os pares com maior similaridade

In [307]:
top_pares_aloj = candidatosAloj.sort_values(by='similaridade_token_sort', ascending=False)

display(top_pares_aloj[[
    'Municipality_listings', 'Municipality_aloj',
    'similaridade_token_sort',
    'similaridade_levenshtein',
    'similaridade_token_set'
]])

,Municipality_listings,Municipality_aloj,similaridade_token_sort,similaridade_levenshtein,similaridade_token_set
471810,Valpaços,VALPAÇOS,1.000000,1.000000,1.000000
0,Valpaços,VALPAÇOS,1.000000,1.000000,1.000000
471790,Moita,MOITA,1.000000,1.000000,1.000000
471789,Barreiro,BARREIRO,1.000000,1.000000,1.000000
471786,Montijo,MONTIJO,1.000000,1.000000,1.000000
...,...,...,...,...,...
82394,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424
88687,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424
95242,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424
75912,Santo Tirso,SANTA CRUZ DA GRACIOSA,0.242424,0.424242,0.242424


> 📊 **Objetivo:** Analisar os melhores pares encontrados entre listings e densidade de alojamento local.  
> Essencial para realizar a correspondência automática entre os datasets.

### ✅ Conclusão da Resolução de Identidade

Com base na estratégia de **blocking por prefixo** e no uso de três métricas complementares de similaridade textual (`token_sort`, `token_set` e `Levenshtein`), foi possível gerar uma lista de pares candidatos com elevada probabilidade de corresponderem à mesma entidade (concelho), mesmo quando os nomes diferem ligeiramente entre datasets.

Este processo de **identity resolution** é fundamental para garantir a correta junção entre os dados dos `listings` e os restantes datasets (`rendasm2`, `densidadePopulacional`, etc.), assegurando que os valores agregados por concelho sejam coerentes e fiáveis.

As próximas etapas poderão incluir:
- Definir um **limiar de corte** para considerar pares como correspondentes (ex: ≥ 0.9 em `token_sort`);
- Realizar a **atribuição de `MunicipalityCode` ao dataset `listings`**, com base nos matches mais prováveis;
- Integrar os datasets agora harmonizados para análise estatística e visualização dos impactos territoriais.

Este processo completa a **Fase 1** do projeto, deixando os dados prontos para análise cruzada, correlações e visualizações interativas.


---

## 🧾 Conclusões e Trabalho Futuro

A Fase 1 do projeto consistiu na importação, profiling, limpeza, preparação e início da integração de múltiplos datasets relacionados com o impacto dos Alojamentos Locais (Airbnb) no mercado habitacional português.

### ✅ Conclusões

- Os dados foram transformados e normalizados de forma a garantir **consistência semântica e estrutural** entre fontes com granularidade e formato distintos;
- Foi adotado um **modelo de integração baseado no concelho (`MunicipalityCode`)**, com o apoio de chaves auxiliares (`Municipality_norm`) para facilitar junções;
- Técnicas de **blocking e similarity metrics** permitiram gerar pares candidatos para a resolução de identidade, essencial para ligar dados do `portugal_listings` (sem `geocod`) aos restantes datasets georreferenciados;
- As três métricas escolhidas (Levenshtein, token_sort, token_set) mostraram-se eficazes na identificação de correspondências mesmo com nomes ambíguos, longos ou com ordem trocada.

### 🔮 Trabalho Futuro

- **Definir limiares de confiança** para aceitar ou rejeitar pares com base nas medidas de similaridade (ex: ≥ 0.90);
- **Atribuir `MunicipalityCode` ao dataset `listings`** com base nos matches validados;
- **Integrar os datasets finais** para construir uma tabela única por concelho com: média de preços, densidade populacional, densidade de alojamentos e valor médio de renda;
- Aplicar **visualizações interativas** (ex: choropleths) para explorar o impacto do alojamento local;
- Avaliar possíveis **correlações** entre as variáveis — por exemplo, se maior densidade de listings está associada a aumento de preços ou densidade populacional.

---

> Esta preparação abre caminho para análises exploratórias, estatísticas e preditivas que respondam à questão central:  
**Qual o impacto da proliferação de Alojamentos Locais no custo e estrutura urbana do mercado de habitação em Portugal?**
